<a href="https://colab.research.google.com/github/sadevans/DL_NLP_ITMO/blob/hw_1/dl_nlp_hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install -q corus
!pip install -q nltk
!pip install pymorphy2
!pip install natasha
!wget -O pymorphy2-dicts-ru.tar.gz https://files.pythonhosted.org/packages/b2/b4/732ff6eeac8c9ea22e7e1c7a321b21b6f3ba19d5e0a8925f35da9c8ebbb2/pymorphy2-dicts-ru-2.4.404381.4453942.tar.gz


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 2.7 MB/s eta 0:00:00
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26097 sha256=0f0ce3dbaeddb9d5db647c1b803075077a24f6f1957c246179fa59fccdeed505
  Stored in directory: /root/.cache/pip/wheels/31/d7/d9/eec6891f78cac19a693bd40ecb8365d2f4613318c145ec9816
Successfully built intervaltree


In [8]:
import os
import random
import gzip
import tarfile
import shutil
import re
import string
import unicodedata
from datetime import datetime as dtm
from collections import OrderedDict
from multiprocessing import Pool, cpu_count

import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MaxAbsScaler, StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report, balanced_accuracy_score, confusion_matrix, f1_score, accuracy_score
from sklearn.dummy import DummyClassifier

from tqdm import tqdm
import pymorphy2
from pymorphy2 import MorphAnalyzer
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from natasha import Segmenter, MorphVocab, Doc
from natasha import NewsEmbedding, NewsMorphTagger, NewsSyntaxParser

import seaborn as sns
import corus
from corus import load_lenta

nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [9]:
seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)

1. Загрузите набор данных lenta-ru-news с помощью библиотеки Corus для задачи классификации текстов по топикам (пригодятся атрибуты title, text, topic)- **1 балл**
2. Подготовьте данные к обучению: - **3 балла**
    - Предобработайте данные: реализуйте оптимальную, на ваш взгляд, предобработку текстов (нормализация, очистка, стемминг/лемматизация и т.п.) и таргета.
    - **hint**: для ускорения обработки  и обучения можно ограничиться не всем датасетом, а его репрезентативной частью, например, размера 100_000.
    - Кратко опишите пайплайн, на котором остановились, и почему.
    - Разделите датасет на обучающую, валидационную и тестовую выборки со стратификацией в пропорции 60/20/20. В качестве целевой переменной используйте атрибут `topic`
3. Замерьте базовое качество с любым dummy-бейзлайном - **0.5 балла**
4. Обучите модель `sklearn.linear_model.LogisticRegression` с двумя вариантами векторизации: **2 балла**
  - `sklearn.feature_extraction.text.CountVectorizer`
  - `sklearn.feature_extraction.text.TfidfVectorizer`
5. Попробуйте улучшить качество, подобрав оптимальные гиперпараметры трансформаций и модели на кросс-валидации **1 балл**
6. Оцените качество лучшего пайплайна на отложенной выборке - **0.5 балла**

**Общее**

- Принимаемые решения обоснованы (почему выбрана определенная архитектура/гиперпараметр/оптимизатор/преобразование и т.п.) - **1 балл**
- Обеспечена воспроизводимость решения: зафиксированы random_state, ноутбук воспроизводится от начала до конца без ошибок - **1 балл**

## Загрузка данных

Загружаю данные и выбираю только столбцы 'title', 'text', 'topic'

In [10]:
def download_data(data_dir='./data', columns2save=['title', 'text', 'topic']):
  if not os.path.exists(f'{data_dir}/lenta-ru-news.csv'):
      os.system(f'mkdir -p {data_dir} && wget -P {data_dir} https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz')

  path = f'{data_dir}/lenta-ru-news.csv.gz'
  records = corus.load_lenta(path)

  data = {
      'title': [],
      'text': [],
      'topic': [],
      'url': [],
      'tags': [],
  }

  for record in records:
      data['title'].append(record.title)
      data['text'].append(record.text)
      data['url'].append(record.url)
      data['tags'].append(record.tags)
      data['topic'].append(record.topic)

  df = pd.DataFrame(data)
  return df[columns2save]

In [11]:
data = download_data()
data.head()

,title,text,topic
0,Названы регионы России с самой высокой смертно...,Вице-премьер по социальным вопросам Татьяна Го...,Россия
1,Австрия не представила доказательств вины росс...,Австрийские правоохранительные органы не предс...,Спорт
2,Обнаружено самое счастливое место на планете,Сотрудники социальной сети Instagram проанализ...,Путешествия
3,В США раскрыли сумму расходов на расследование...,С начала расследования российского вмешательст...,Мир
4,Хакеры рассказали о планах Великобритании зами...,Хакерская группировка Anonymous опубликовала н...,Мир


## Подготовка данных к обучению

Выбираю только топики, у которых кол-во записей больше 500


### сэмплирование данных

In [12]:
data['topic'].value_counts()

,count
topic,
Россия,160519
Мир,136680
Экономика,79538
Спорт,64421
Культура,53803
Бывший СССР,53402
Наука и техника,53136
Интернет и СМИ,44675
Из жизни,27611


In [13]:
topics_to_use = data['topic'].value_counts()[data['topic'].value_counts() > 500].index

data = data[((data['topic']!= '') & (data['topic'].isin(topics_to_use)))]
print('Len dataset: ', len(data))
data['topic'].value_counts(normalize=True)

Len dataset:  738622


,proportion
topic,
Россия,0.217322
Мир,0.185047
Экономика,0.107684
Спорт,0.087218
Культура,0.072842
Бывший СССР,0.072299
Наука и техника,0.071939
Интернет и СМИ,0.060484
Из жизни,0.037382


In [14]:
def sample_data(data, target_col='topic', n_samples=100000):
  class_counts = data[target_col].value_counts()
  samples_per_class = (class_counts / class_counts.sum() * n_samples).astype(int)

  # samples_per_class = samples_per_class.apply(lambda x: max(x, min_samples_per_class))

  sampled_data = pd.DataFrame()
  for class_label, n in samples_per_class.items():
      class_sample = data[data[target_col] == class_label].sample(
          n=min(n, len(data[data[target_col] == class_label])),
          random_state=seed_value
      )
      sampled_data = pd.concat([sampled_data, class_sample])

  sampled_data = sampled_data.sample(frac=1, random_state=seed_value).reset_index(drop=True)

  print(sampled_data[target_col].value_counts(normalize=True))
  return sampled_data

In [15]:
sampled_data = sample_data(data)

topic
Россия               0.217337
Мир                  0.185055
Экономика            0.107689
Спорт                0.087217
Культура             0.072846
Бывший СССР          0.072296
Наука и техника      0.071936
Интернет и СМИ       0.060485
Из жизни             0.037383
Дом                  0.029422
Силовые структуры    0.026532
Ценности             0.010511
Бизнес               0.010011
Путешествия          0.008671
69-я параллель       0.001710
Крым                 0.000900
Name: proportion, dtype: float64


In [16]:
sampled_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99992 entries, 0 to 99991
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   99992 non-null  object
 1   text    99992 non-null  object
 2   topic   99992 non-null  object
dtypes: object(3)
memory usage: 2.3+ MB


Всего получилось 99 992 экземпляров

### предобработка данных

Остановилась на следующем пайплайне обработки данных:
1. привожу текст к нижнему регистру
2. удаляю все символе, кроме букв и символов других языков, также осталвяю тире и проблеы, потому что могут быть слова, написанные через дефис или тире
Удаляю еще всякие знаки разделения, отличные от пробелов, и непонятные юникод символы.
3. токенизирую текст
4. избавляюсь от стоп слов на русском и лемматизирую текст
5. соединяю лемматизированные токены обратно в предложение

In [17]:
sampled_data['content'] = sampled_data['title'] + '. ' + sampled_data['text']

In [18]:
def get_russian_alphabet():
    return ''.join([chr(i) for i in range(ord('а'), ord('я') + 1)] + ['ё'])

In [19]:
def find_all_chars(df, column='text'):
    chars = set()
    for item in df[column]:
        chars = chars.union(item)
    chars = sorted(chars)
    return chars

In [20]:
def find_review_by_character(df, char_, max_len=500, column='text'):
    for review in df[column]:
        if char_ in review and len(review) < max_len:
            return review
        else:
          continue

Посмотрю, какие символы присутвуют, чтобы понять, от чего чистить текст

In [21]:
all_chars = find_all_chars(sampled_data, column='content')
print(all_chars)

['\x08', '\t', '\x13', '\x1d', '\x1e', '\x1f', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '\x7f', '\x98', '\xa0', '¡', '£', '¥', '©', '«', '\xad', '®', '°', '±', '·', '»', '¼', '¾', 'À', 'Á', 'Å', 'Ç', 'É', 'Ô', 'Ö', '×', 'Ü', 'ß', 'à', 'á', 'â', 'ã', 'ä', 'å', 'æ', 'ç', 'è', 'é', 'ê', 'ë', 'ì', 'í', 'î', 'ï', 'ð', 'ñ', 'ò', 'ó', 'ô', 'ö', '÷', 'ø', 'ú', 'û', 'ü', 'ý', 'ā', 'ă', 'ą', 'Ć', 'ć', 'Č', 'č', 'Đ', 'ē', 'ė', 'ę', 'ě', 'ğ', 'ī', 'İ', 'ı', 'Ł', 'ł', 'ń', 'ņ', 'ō', 'ő', 'œ', 'ś', 'ş', 'Š', 'š', 'ū', 'ů', 'ź', 'ż', 'ʹ', 'ʺ', 'ʼ', '́', '̆', 'Β', 'Λ', 'Φ', 

надо будет удалить эмоджи, всякие знаки препинания, цифры и непонятные символы

еще нужно посмотреть, что за тексты с нерусскими буквами

In [15]:
for c in all_chars:
    print(f'reviews for char = {c}: \n')
    print(find_review_by_character(sampled_data, c, max_len=1000, column='content'))
    print()

reviews for char = : 

None

reviews for char = 	: 

Названы самые популярные направления у россиян на июньские праздники. Самым популярным направлением для отдыха на июньские праздники среди россиян, так же как и в прошлом году, стала Барселона. Об этом сообщается в пресс-релизе туристического сервиса KAYAK.ru, поступившем в редакцию «Ленты.ру». Отмечается, что при составлении топ-листа учитывались билеты, купленные на период с 11 по 14 июня. Второе место в рейтинге занял Мюнхен, третье — Адлер. Отмечается, что в 2014 году это были Санкт-Петербург и Стамбул соответственно. Список десяти самых популярных туристических направлений на июньские праздники по версии KAYAK.ru выглядит так: 1.	Барселона2.	Мюнхен3.	Адлер4.	Дюссельдорф5.	Рим6.	Прага7.	Стамбул8.	Санкт-Петербург9.	Париж10.	Нью-Йорк Ранее стало известно, что число российских туристов за границей в первом квартале 2015 года сократилось на 40 процентов по сравнению с аналогичным периодом прошлого года. Всего на отдых из России выех

Это всякие хэштеги, имена пользователей, именна газет/таблоидов и брендов

думаю, что оставлю названия, тк это может быть критично для новости

In [81]:
def clean_text(text):
    if isinstance(text, list):
      text = text[0]
    if not isinstance(text, str):
        return ''

    lemmas = []
    text = text.lower()

    allowed_chars = set(russian_alphabet + string.ascii_lowercase + ' -')

    text = ''.join(char if (char in allowed_chars or unicodedata.category(char).startswith('L')) else ' ' for char in text)
    text = re.sub(r'\s+', ' ', text).strip()
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)

    for token in doc.tokens:
      token.lemmatize(morph_vocab)
      if token.text not in stopwords_list and token.text != '-':
        lemmas.append(token.lemma)
    return ' '.join(lemmas)

In [82]:
russian_alphabet = get_russian_alphabet()
stopwords_list = set(stopwords.words('russian'))

segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

In [83]:
%%time
with Pool(cpu_count()) as pool:
        processed_content = pool.map(clean_text, sampled_data['content'].tolist())

sampled_data['processed_content'] = processed_content

CPU times: user 17.5 s, sys: 4.16 s, total: 21.6 s
Wall time: 1h 3min 14s


In [84]:
sampled_data['processed_content']

,processed_content
0,сирия снайпер открыть огонь похоронный процесс...
1,ультраправый партия автор фитны пройти европар...
2,прораб арестовать падение башенный кран прораб...
3,милиционер спасатель найти взрывчатка площадь ...
4,кадыров выступить против проведение выпускной ...
...,...
99987,первый дело похищение человек чечня дойти евро...
99988,мутко оценить готовность россия кубок конфедер...
99989,предсказать глобальный нехватка питьевой вода ...
99990,московский школьница снять порча тысяча доллар...


Посмотрим, какие символы еще остались

In [85]:
all_chars_left = find_all_chars(sampled_data, column='processed_content')
print(all_chars_left)

[' ', '-', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ß', 'à', 'á', 'â', 'ã', 'ä', 'å', 'æ', 'ç', 'è', 'é', 'ê', 'ë', 'ì', 'í', 'î', 'ï', 'ð', 'ñ', 'ò', 'ó', 'ô', 'ö', 'ø', 'ú', 'û', 'ü', 'ý', 'ā', 'ă', 'ą', 'ć', 'č', 'đ', 'ē', 'ė', 'ę', 'ě', 'ğ', 'ī', 'ı', 'ł', 'ń', 'ņ', 'ō', 'ő', 'œ', 'ś', 'ş', 'š', 'ū', 'ů', 'ź', 'ż', 'ʹ', 'ʺ', 'ʼ', 'α', 'β', 'ζ', 'ι', 'λ', 'μ', 'π', 'ς', 'σ', 'φ', 'ω', 'ό', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я', 'є', 'і', 'ї', 'ј', 'љ', 'ћ', 'ә', 'ӣ', 'ก', 'ข', 'ค', 'ง', 'จ', 'ช', 'ณ', 'ต', 'ถ', 'ธ', 'น', 'บ', 'ป', 'พ', 'ฟ', 'ภ', 'ม', 'ย', 'ร', 'ล', 'ว', 'ศ', 'อ', 'ะ', 'า', 'เ', 'ไ', 'ἅ', 'あ', 'い', 'う', 'え', 'か', 'が', 'き', 'く', 'ぐ', 'こ', 'さ', 'し', 'じ', 'す', 'そ', 'た', 'っ', 'つ', 'て', 'で', 'と', 'ど', 'な', 'に', 'ね', 'の', 'は', 'ふ', 'ま', 'み', 'む', 'め', 'も', 'ゃ', 'よ',

In [86]:
sampled_data['processed_content'].iloc[0]

'сирия снайпер открыть огонь похоронный процессия столица сирия дамаск снайпер открыть огонь участник похоронный шествие честь погибнуть столкновение полиция сообщать agence france-presse данные аль-джазира пригород дамаск дума убить четыре человек сведение reuters свидетельствовать район сирийский столица барзех также погибнуть человек участие шествие принять несколько десяток тысяча человек выступление посвятить память погибнуть время акция протест пятница апрель аналогичный шествие состояться юг сирия город эзрех власть также применить сила открыть огонь группа человек собираться присоединиться процессия погибнуть человек большинство свой участник похоронный процессия являться сторонник оппозиция выступать против режим президент сирия башар асад стоить отметить апрель асад отменить действовать страна протяжение год режим чрезвычайный ситуация ранее провести ряд кадровый перестановка правительство'

In [87]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

sampled_data['topic_encoded'] = le.fit_transform(sampled_data['topic'])
target_names = le.classes_

### разделение на выборки

Делаю стратификацию по целевой переменной

In [88]:
sampled_data.to_csv('./sampled_data_processed.csv', index=False)

In [89]:
X_train, X_test_val, y_train, y_test_val = train_test_split(
    sampled_data['processed_content'],
    sampled_data['topic_encoded'],
    test_size=0.4,
    stratify=sampled_data['topic_encoded'],
    random_state=seed_value,
)
X_val, X_test, y_val, y_test = train_test_split(X_test_val, y_test_val, test_size=0.5, stratify=y_test_val, random_state=seed_value)

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

X_train shape: (59995,), y_train shape: (59995,)
X_val shape: (19998,), y_val shape: (19998,)
X_test shape: (19999,), y_test shape: (19999,)


## Базовое качество с dummy-бейзлайном


In [90]:
vectorizer = CountVectorizer(max_df=0.7, min_df=0.003)
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

In [91]:
vectorizer.get_feature_names_out()

array(['afp', 'agence', 'air', ..., 'яркий', 'ясный', 'ящик'],
      dtype=object)

In [92]:
dummy_clf = DummyClassifier(strategy="most_frequent").fit(X_train_vectorized, y_train)
preds = dummy_clf.predict(X_test_vectorized)
print(classification_report(y_test, preds, zero_division=0))
print(f"Dummy Classifier F1-score (macro): {f1_score(y_test, preds, average='macro'):.4f}")

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        34
           1       0.00      0.00      0.00       200
           2       0.00      0.00      0.00      1446
           3       0.00      0.00      0.00       588
           4       0.00      0.00      0.00       747
           5       0.00      0.00      0.00      1210
           6       0.00      0.00      0.00        18
           7       0.00      0.00      0.00      1457
           8       0.00      0.00      0.00      3701
           9       0.00      0.00      0.00      1439
          10       0.00      0.00      0.00       173
          11       0.22      1.00      0.36      4347
          12       0.00      0.00      0.00       530
          13       0.00      0.00      0.00      1745
          14       0.00      0.00      0.00       210
          15       0.00      0.00      0.00      2154

    accuracy                           0.22     19999
   macro avg       0.01   

## Обучение модели логистической регрессии

### с CountVectorizer

In [94]:
vec = CountVectorizer(max_features=100000, ngram_range=(1, 1))

train_vectors = vec.fit_transform(X_train)
val_vectorts = vec.transform(X_val)
test_vectorts = vec.transform(X_test)


scaler = MaxAbsScaler()
train_vectors = scaler.fit_transform(train_vectors)
val_vectorts = scaler.transform(val_vectorts)
test_vectorts = scaler.transform(test_vectorts)


clf = LogisticRegression(max_iter=1500, random_state=seed_value, class_weight='balanced')
clf.fit(train_vectors, y_train)
test_pred = clf.predict(test_vectorts)

print(classification_report(y_test, test_pred))
test_accuracy = accuracy_score(y_test, test_pred)
print(f"Качество на тесте (CountVectorizer): {test_accuracy:.4f}")

              precision    recall  f1-score   support

           0       0.75      0.35      0.48        34
           1       0.52      0.45      0.48       200
           2       0.82      0.87      0.85      1446
           3       0.80      0.86      0.83       588
           4       0.55      0.65      0.60       747
           5       0.75      0.76      0.76      1210
           6       0.62      0.44      0.52        18
           7       0.86      0.89      0.88      1457
           8       0.81      0.81      0.81      3701
           9       0.84      0.85      0.85      1439
          10       0.64      0.72      0.68       173
          11       0.83      0.77      0.80      4347
          12       0.58      0.63      0.60       530
          13       0.96      0.96      0.96      1745
          14       0.91      0.79      0.84       210
          15       0.85      0.85      0.85      2154

    accuracy                           0.81     19999
   macro avg       0.76   

### с TFIDFVectorizer

In [97]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000, ngram_range=(1, 2))),
    ('clf', LogisticRegression(max_iter=1500, random_state=seed_value, class_weight='balanced'))
])

In [98]:
pipeline.fit(X_train, y_train)

y_pred_test = pipeline.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred_test)
print(f"Качество на тесте (TfidfVectorizer): {test_accuracy:.4f}")
print("Отчет по классификации на тесте:")
print(classification_report(y_test, y_pred_test))

Качество на тесте (TfidfVectorizer): 0.7853
Отчет по классификации на тесте:
              precision    recall  f1-score   support

           0       0.27      0.35      0.31        34
           1       0.34      0.69      0.45       200
           2       0.78      0.87      0.82      1446
           3       0.73      0.88      0.79       588
           4       0.49      0.69      0.57       747
           5       0.72      0.78      0.75      1210
           6       0.21      0.67      0.32        18
           7       0.86      0.88      0.87      1457
           8       0.83      0.76      0.80      3701
           9       0.83      0.81      0.82      1439
          10       0.46      0.80      0.58       173
          11       0.87      0.68      0.76      4347
          12       0.45      0.74      0.56       530
          13       0.96      0.95      0.96      1745
          14       0.79      0.84      0.82       210
          15       0.86      0.78      0.82      2154

   

Лучшим получился пайплайн с CountVectorizer